---

<div style="display: flex; align-items: center; justify-content: space-between;">
  <img src="../../.images/PNAV-logo.png" alt="Logo del PNAV" style="width: auto; max-height: 100px;">
  <img src="../../.images/MITECO-logo_background.png" alt="Logo del MITECO" style="width: auto; max-height: 100px;">
</div>

---

# Descarga de datos geográficos del Geoserver del IEPNB
A continuación, se describirá como descargar datos **geográficos** del Inventario Español del Patrimonio Natural y de la Biodiversidad (IEPNB).
Para una información más detallada y completa, consultar:
- Sección de [servicios interoperables del IEPNB](https://iepnb.gob.es/recursos/servicios-interoperables)
- Portal del [Geoserver del IEPNB](https://geoserver-des.iepnb.es/geoserver/web/)

## 1. Instalación de módulos y librerías
Se instalan los módulos y librerías necesarios para ejecutar el código.

In [2]:
!pip install owslib

## 2. Definición de variables
Se guarda en un array los nombre de las secciones que se pueden consultar. Las secciones disponibles son:
- cartografia_base
- ecosistemas
- alertas_anuales_vegetacion
- EIKOS_Alertas
- cambios_anuale
- vegetacion
- areas_marinas_protegidas
- ENP
- RN2000
- convenio_internacional
- espacios_protegidos
- especies
- foto_fija_mfe
- incendios_forestales
- vias_pecuarias
- recursos_geneticos
- INES

Para más información sobre las secciones y como acceder a ellas, consultar el apartado [Servicios Web Mapas del IEPNB](https://iepnb.gob.es/recursos/servicios-interoperables)

In [8]:
sections = ['cartografia_base','ecosistemas','alertas_anuales_vegetacion','EIKOS_Alertas','cambios_anuales_vegetacion','areas_marinas_protegidas','ENP','RN2000','convenio_internacional','espacios_protegidos','especies','foto_fija_mfe','incendios_forestales','vias_pecuarias','recursos_geneticos','INES']

## 3. Peticiones

### 3.1. Importación de módulos y librerías
Se importan los módulos y librerías necesarios para la ejecución del código.

In [2]:
import requests
import folium
import re
import os

### 3.2. Definición de funciones
#### 3.2.1. Función "get_subsections"
Dado el nombre de la sección de los recursos geográficos del IEPNB devuelve en un array todas las subsecciones que tiene.

In [3]:
def get_subsections(section):
    url = 'https://geoserver-des.iepnb.es/geoserver/'+section+'/wms?service=WMS&request=GetCapabilities'

    response = requests.get(url)

    # Definir el patrón para encontrar contenido desde <Name> hasta </Name>
    pattern = r'<Name>.*?</Name>'

    matches = re.findall(pattern, response.text, re.DOTALL)

    subsections = []
    # Imprimir los resultados completos
    for match in matches:
        subsections.append(match.replace('<Name>','').replace('</Name>',''))

    unique_subsections = list(set(subsections))
    unique_subsections.remove('WMS')
    return unique_subsections

#### 3.2.1. Función "get_subsection_data"
Dado el nombre de la subsección de los recursos geográficos del IEPNB, hace una petición al "geoserver" y devuelve los datos.

In [4]:
def get_subsection_data(output_dir, section, subsection):
    filename = output_dir+section+'_'+subsection+'.geojson.html'

    # URL base del servicio WFS
    url = "https://geoserver.iepnb.es/geoserver/"+section+"/ows"

    # Parámetros de la solicitud
    params = {
            'service': 'WFS',
            'version': '1.0.0',
            'request': 'GetFeature',
            'typeName': section+':'+subsection,
            'maxFeatures': 50,
            'outputFormat': 'application/json'
    }

    # Realizar la solicitud GET con los parámetros
    response = requests.get(url, params=params)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        try:
            # Intentar convertir la respuesta a JSON
            geojson_data = response.json()

            # Crear un mapa centrado en un punto (puedes ajustar las coordenadas)
            mapa = folium.Map(location=[40.0, -3.7], zoom_start=6)

            # Añadir los datos GeoJSON al mapa
            folium.GeoJson(geojson_data).add_to(mapa)

            # Guardar el mapa como archivo HTML
            mapa.save(filename)
            
        except requests.exceptions.JSONDecodeError as e:
            print(f"        - Error al decodificar JSON para {subsection}: {e}")
    else:
        print(f"        - Error: {response.status_code}")

### 3.3. Ejecución
Se ejecutan las peticiones de todas las secciones y subsecciones.
Se guardan los documentos en el directorio geojson/.

In [ ]:
i = 1
for section in sections:
    print(f'Sección {i}/{len(sections)}: {section}')
    output_dir = 'geojson/'+section+'/'
    os.makedirs(output_dir, exist_ok=True)
    unique_subsections= get_subsections(section)
    j=1
    for subsection in unique_subsections:
        print(f'    - Subsección {j}/{len(unique_subsections)}: {subsection}')
        get_subsection_data(output_dir, section, subsection)
        j += 1
    i += 1